# Tennis Match Predictor

Author: David Hidalgo Fàbregas  
Date: 31/01/2026

## Libraries

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

## 1. Elo System

The official ATP ranking sometimes takes time to reflect a player's current form, while a dynamic Elo rating adapts more quickly.

In [2]:
# Load data
matches = pd.read_csv('../Dataset/atp_matches_till_2022.csv')

# Chronological order (CRUCIAL for Elo)
matches['tourney_date'] = pd.to_datetime(matches['tourney_date'], format='%Y%m%d', errors='coerce')
matches = matches.sort_values('tourney_date').reset_index(drop=True)

# Elo configuration
elo_ratings = {}
k_factor = 32  # How much does the score change per game?

def get_elo(player_id):
    return elo_ratings.get(player_id, 1500) # 1500 is the base score

winner_elo_list = []
loser_elo_list = []

for idx, row in matches.iterrows():
    w_id = row['winner_id']
    l_id = row['loser_id']
    
    w_elo = get_elo(w_id)
    l_elo = get_elo(l_id)
    
    # We save the Elo rating BEFORE the match (this is what the model will use).
    winner_elo_list.append(w_elo)
    loser_elo_list.append(l_elo)
    
    # Compute expected probabilities
    expected_w = 1 / (1 + 10 ** ((l_elo - w_elo) / 400))
    expected_l = 1 / (1 + 10 ** ((w_elo - l_elo) / 400))
    
    # Update Elos
    elo_ratings[w_id] = w_elo + k_factor * (1 - expected_w)
    elo_ratings[l_id] = l_elo + k_factor * (0 - expected_l)

# Add to dataframe
matches['w_elo'] = winner_elo_list
matches['l_elo'] = loser_elo_list

In [3]:
matches.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,w_elo,l_elo
0,1968-9343,Bloemfontein,Hard,64,A,1968-01-08,262,209426,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.0,1500.0
1,1968-9343,Bloemfontein,Hard,64,A,1968-01-08,270,100011,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.0,1500.0
2,1968-9343,Bloemfontein,Hard,64,A,1968-01-08,271,109914,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.0,1500.0
3,1968-9343,Bloemfontein,Hard,64,A,1968-01-08,272,100060,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.0,1516.0
4,1968-9343,Bloemfontein,Hard,64,A,1968-01-08,273,100137,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1500.0,1500.0


In [4]:
matches.describe()

,draw_size,tourney_date,match_num,winner_id,winner_seed,winner_ht,winner_age,loser_id,loser_seed,loser_ht,...,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,w_elo,l_elo
count,188161.000000,188161,188161.000000,188161.000000,69694.000000,171924.000000,186826.000000,188161.000000,35337.000000,159463.000000,...,92219.000000,92220.000000,92219.000000,92219.000000,153197.000000,105973.000000,144834.000000,104354.000000,188161.000000,188161.000000
mean,52.926292,1993-09-29 06:54:48.082014720,76.618598,103820.251673,6.280225,184.449187,25.658362,104967.986995,7.667402,184.226592,...,14.985263,12.197387,4.812002,8.742884,75.255716,1366.471611,112.884150,859.219896,1749.346554,1661.260153
min,2.000000,1968-01-08 00:00:00,1.000000,100001.000000,1.000000,160.000000,14.300000,100001.000000,1.000000,160.000000,...,0.000000,0.000000,-6.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1189.875892,1135.813090
25%,32.000000,1980-04-14 00:00:00,10.000000,100402.000000,2.000000,180.000000,22.700000,100502.000000,4.000000,180.000000,...,10.000000,9.000000,2.000000,6.000000,17.000000,489.000000,37.000000,361.000000,1607.667051,1534.246274
50%,32.000000,1993-03-01 00:00:00,25.000000,101686.000000,5.000000,185.000000,25.200000,101843.000000,6.000000,185.000000,...,14.000000,11.000000,4.000000,8.000000,44.000000,846.000000,70.000000,630.000000,1730.082130,1640.774636
75%,64.000000,2006-07-21 00:00:00,80.000000,103898.000000,8.000000,188.000000,28.200000,104252.000000,10.000000,188.000000,...,19.000000,15.000000,7.000000,11.000000,86.000000,1532.000000,118.000000,1013.000000,1865.239757,1755.027710
max,128.000000,2022-11-27 00:00:00,1701.000000,211468.000000,35.000000,211.000000,58.700000,211805.000000,35.000000,211.000000,...,101.000000,91.000000,28.000000,38.000000,2101.000000,16950.000000,2159.000000,16950.000000,2472.711632,2473.611453
std,36.446303,NaN,110.714957,11470.048991,5.509548,6.667033,4.045128,14866.251405,5.995551,6.655036,...,7.220377,4.129834,3.275387,4.131839,121.053512,1726.089469,162.191701,987.192154,186.335818,152.596527


In [5]:
matches['tourney_level'].value_counts()

tourney_level
A    123868
G     26521
M     22913
D     14310
F       549
Name: count, dtype: int64

## 2. Feature Selection
Only data known before the game can be used.  
Do not use statistics from the current game (such as w_ace or minutes from the same game), but historical averages can be used.
- Ranking difference: `Rank_A-Rank_B`. Logarithm might be useful because the difference between `#1` and `#5` is more significative than `#100` and `#105`.
- Elo difference: `Elo_A-Elo_B`.
- Surface performance: winning percentage of the player in this surface in the last year.
- Head-to-Head history: How many times A has won against B in the past.
- Exhaustion: How many matches did the player play in the last 2 weeks. How many minuts did the player play in the last match?
- Phisical Data: Heigh difference, dominant hand...

In [6]:
# Round map
round_map = {
    'R128': 1,
    'R64': 2,
    'R32': 3,
    'R16': 4,
    'QF': 5,
    'SF': 6,
    'BR': 6.5, # Bronze Medal (Third-place play-off between SF and Final)
    'F': 7,
    'RR': 8,   # Round Robin (It is usually the ATP Finals, a very high level)
    'ER': 1    # Early Round (It is usually used in the Davis Cup or very low rounds, minimum value)
}

# Tourney level map
level_map = {
    'G': 5,  # Grand Slam
    'F': 4,  # Tour Finals
    'M': 3,  # Masters 1000
    'A': 2,  # ATP 250/500
    'D': 2,  # Davis Cup (Level similar to average ATP in difficulty)
    'C': 1,  # Challengers
    'S': 1   # Satellites/Futures
}

In [11]:
WINDOW_SIZE = 10  # Last N matches for overall form (you can try 5 or 10)

history_general = {}
history_h2h = {}

# We define the metrics we are going to save for each match.
stats_cols = [
    'ace',
    'df',
    'svpt',
    '1stIn',
    '1stWon',
    '2ndWon',
    'SvGms',
    'bpSaved',
    'bpFaced'
]

In [12]:
def update_history(history_dict, key, new_stats, max_len=None):
    """
    Add statistics to the history by managing the list manually.
    """
    if key not in history_dict:
        history_dict[key] = []

    history_dict[key].append(new_stats)
    if max_len is not None and len(history_dict[key])>max_len:
        history_dict[key].pop(0)

In [13]:
def get_avg_stats(history_list):
    """Calculate the average of the statistics stored in a list."""
    if not history_list:
        return {} # Returns empty if there is no history
    
    # We transpose the dictionary list to calculate averages by key.
    avgs = {}
    keys = history_list[0].keys()
    
    for k in keys:
        values = [match[k] for match in history_list if match[k] is not np.nan]
        if values:
            avgs[f"avg_{k}"] = np.mean(values)
        else:
            avgs[f"avg_{k}"] = 0
            
    # --- CALCULATION OF DERIVATIVE PERCENTAGES ---
    # It is best to average the resulting percentages.
    if avgs.get('avg_svpt', 0) > 0:
        # % 1st service
        avgs['pct_1stIn'] = avgs.get('avg_1stIn', 0) / avgs.get('avg_svpt')
        # % Points won on first serve (out of those played)
        avgs['pct_1stWon'] = avgs.get('avg_1stWon', 0) / avgs.get('avg_1stIn') if avgs.get('avg_1stIn', 0) > 0 else 0
        # % Points won on second serve
        avgs['pct_2ndWon'] = avgs.get('avg_2ndWon', 0) / (avgs.get('avg_svpt') - avgs.get('avg_1stIn')) if (avgs.get('avg_svpt') - avgs.get('avg_1stIn')) > 0 else 0
        
    return avgs

### Restructure DataFrame - Player A vs Player B

The data has columns named winner_name and loser_name.

The common mistake: If model is trained with these columns, it will quickly learn that te person in the "Winner" column always wins.

**Solution**: Transform each row into a neutral matchup, Player A vs Player B.

The target variable will be: Did player A win? 1 if yes, 0 if no.

It must be randomized who is Player A and who is Player B in each row so that your dataset has a 50/50 balance of wins and losses.

In [16]:
dataset = []

for idx, row in matches.iterrows():
    # We randomly decide whether P1 corresponds to the winner or the loser.
    if random.random() > 0.5:
        # Case: P1 wins
        p1 = {'id': row['winner_id'],
              'elo': row['w_elo'],
              'atp_rank': row['winner_rank'],
              'atp_points': row['winner_rank_points'],
              'age': row['winner_age'],
              'hand': row['winner_hand'],
              'height': row['winner_ht'],
              'ioc': row['winner_ioc']}
        p2 = {'id': row['loser_id'],
              'elo': row['l_elo'],
              'atp_rank': row['loser_rank'],
              'atp_points': row['loser_rank_points'],
              'age': row['loser_age'],
              'hand': row['loser_hand'],
              'height': row['loser_ht'],
              'ioc': row['loser_ioc']}
        target = 1
    else:
        # Case: P1 loses
        p1 = {'id': row['loser_id'],
              'elo': row['l_elo'],
              'atp_rank': row['loser_rank'],
              'atp_points': row['loser_rank_points'],
              'age': row['loser_age'],
              'hand': row['loser_hand'],
              'height': row['loser_ht'],
              'ioc': row['loser_ioc']}
        p2 = {'id': row['winner_id'],
              'elo': row['w_elo'],
              'atp_rank': row['winner_rank'],
              'atp_points': row['winner_rank_points'],
              'age': row['winner_age'],
              'hand': row['winner_hand'],
              'height': row['winner_ht'],
              'ioc': row['winner_ioc']}
        target = 0
        
    # FEATURE ENGINEERING
    # Hand: 1 if left-handed (L), 0 if right-handed (R).
    if p1['hand'] == 'L':
        p1_lefty = 1 
    elif p1['hand'] == 'R':
        p1_lefty = 0
    else:
        p1_lefty = np.nan

    if p2['hand'] == 'L':
        p2_lefty = 1 
    elif p1['hand'] == 'R':
        p2_lefty = 0
    else:
        p2_lefty = np.nan
        
    # Same country
    p1_country = p1['ioc']
    p2_country = p2['ioc']
    is_same_country = 1 if (p1_country == p2_country) else 0

    # Round of the match
    round_name = row['round']
    round_value = round_map.get(round_name)

    # Tourney level
    tl_name = row['tourney_level']
    tl_value = level_map.get(tl_name)

    
    # History
    p1_id = p1['id']
    p2_id = p2['id']

    h2h_key = tuple(sorted([p1_id, p2_id]))
    matchup_history = history_h2h.get(h2h_key, {})


    p1_history_list = history_general.get(p1_id, [])
    p2_history_list = history_general.get(p2_id, [])
    p1_h2h_list = matchup_history.get(p1_id, [])
    p2_h2h_list = matchup_history.get(p2_id, [])
    
    p1_stats = get_avg_stats(p1_history_list)
    p2_stats = get_avg_stats(p2_history_list)
    p1_h2h_stats = get_avg_stats(p1_h2h_list)
    p2_h2h_stats = get_avg_stats(p2_h2h_list)

    # Dataset
    dataset.append({
        'date': row['tourney_date'],

        # P1 vs P2
        'diff_elo': p1['elo'] - p2['elo'],                      # P1 better than P2?
        'diff_rank': p1['atp_rank'] - p2['atp_rank'],           # ATP ranking difference
        'diff_points': p1['atp_points'] - p2['atp_points'],     # ATP points difference
        'diff_age': p1['age'] - p2['age'],                      # P1 older?
        'diff_height': p1['height']-p2['height'],               # P1 taller?
        'p1_is_lefty': p1_lefty,
        'p2_is_lefty': p2_lefty,
        'same_country': is_same_country,

        # Tournament
        'surface': row['surface'],  # Needs One-Hot Encoding later
        'draw_size': row['draw_size'],
        'round': round_value,
        'best_of': row['best_of'],
        'tourney_level': tl_value,

        # Stats P1
        'p1_ace': p1_stats.get('avg_ace', 0),
        'p1_df': p1_stats.get('avg_df', 0),
        'p1_1stWon': p1_stats.get('pct_1stWon', 0),
        'p1_bpSaved': p1_stats.get('avg_bpSaved', 0),

        # Stats P1
        'p2_ace': p2_stats.get('avg_ace', 0),
        'p2_df': p2_stats.get('avg_df', 0),
        'p2_1stWon': p2_stats.get('pct_1stWon', 0),
        'p2_bpSaved': p2_stats.get('avg_bpSaved', 0),

        # H2H History
        'h2h_matches': len(p1_h2h_list), # Number of times they play each other
        'p1_h2h_ace': p1_h2h_stats.get('avg_ace', 0),
        'p1_h2h_df': p1_h2h_stats.get('avg_df', 0),
        'p1_h2h_1stWon': p1_h2h_stats.get('pct_1stWon', 0),
        'p1_h2h_bpSaved': p1_h2h_stats.get('avg_bpSaved', 0),
        'p2_h2h_ace': p2_h2h_stats.get('avg_ace', 0),
        'p2_h2h_df': p2_h2h_stats.get('avg_df', 0),
        'p2_h2h_1stWon': p2_h2h_stats.get('pct_1stWon', 0),
        'p2_h2h_bpSaved': p2_h2h_stats.get('avg_bpSaved', 0),

        # Target
        'target': target
    })

    # --- UPDATE HISTORY AFTER THE MATCH ---
    if row['winner_id'] == p1_id:
        # P1 Won
        stats_p1_now = {k: row[f'w_{k}'] for k in stats_cols} # Ej: row['w_ace']
        stats_p2_now = {k: row[f'l_{k}'] for k in stats_cols} # Ej: row['l_ace']
    else:
        # P1 Lost
        stats_p1_now = {k: row[f'l_{k}'] for k in stats_cols}
        stats_p2_now = {k: row[f'w_{k}'] for k in stats_cols}
        
    # Save in general
    update_history(history_general, p1_id, stats_p1_now, max_len=WINDOW_SIZE)
    update_history(history_general, p2_id, stats_p2_now, max_len=WINDOW_SIZE)
    
    # Save in H2H (Guardamos ambos lados para tener el promedio del partido)
    # Ojo: Para H2H a veces interesa saber "cómo saca P1 contra P2". 
    # Para simplificar, aquí guardamos el promedio del partido o podrias guardar 
    # dos listas separadas en el diccionario h2h.
    if h2h_key not in history_h2h:
        history_h2h[h2h_key] = {
            p1_id: [],
            p2_id: []
        }
    if p1_id not in history_h2h[h2h_key]: history_h2h[h2h_key][p1_id] = []
    if p2_id not in history_h2h[h2h_key]: history_h2h[h2h_key][p2_id] = []

    history_h2h[h2h_key][p1_id].append(stats_p1_now)
    history_h2h[h2h_key][p2_id].append(stats_p2_now)

df_train = pd.DataFrame(dataset)
df_train["p1_is_lefty"] = df_train["p1_is_lefty"].astype('Int64')
df_train["p2_is_lefty"] = df_train["p2_is_lefty"].astype('Int64')

# 5. One Hot Encoding for surface (Clay, Grass, Hard)
df_train = pd.get_dummies(df_train, columns=['surface'], drop_first=True)
df_train = df_train.sort_values('date').reset_index(drop=True)

In [17]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188161 entries, 0 to 188160
Data columns (total 34 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            188161 non-null  datetime64[ns]
 1   diff_elo        188161 non-null  float64       
 2   diff_rank       140921 non-null  float64       
 3   diff_points     103631 non-null  float64       
 4   diff_age        182601 non-null  float64       
 5   diff_height     153371 non-null  float64       
 6   p1_is_lefty     184156 non-null  Int64         
 7   p2_is_lefty     161689 non-null  Int64         
 8   same_country    188161 non-null  int64         
 9   draw_size       188161 non-null  int64         
 10  round           188161 non-null  float64       
 11  best_of         188161 non-null  int64         
 12  tourney_level   188161 non-null  int64         
 13  p1_ace          68789 non-null   float64       
 14  p1_df           68789 non-null   flo

In [18]:
df_train.head()

,date,diff_elo,diff_rank,diff_points,diff_age,diff_height,p1_is_lefty,p2_is_lefty,same_country,draw_size,...,p1_h2h_1stWon,p1_h2h_bpSaved,p2_h2h_ace,p2_h2h_df,p2_h2h_1stWon,p2_h2h_bpSaved,target,surface_Clay,surface_Grass,surface_Hard
0,1968-01-08,0.0,NaN,NaN,NaN,NaN,<NA>,<NA>,1,64,...,0.0,0.0,0.0,0.0,0.0,0.0,1,False,False,True
1,1968-01-08,16.0,NaN,NaN,NaN,NaN,<NA>,<NA>,1,64,...,0.0,0.0,0.0,0.0,0.0,0.0,0,False,False,True
2,1968-01-08,0.0,NaN,NaN,NaN,NaN,<NA>,<NA>,1,64,...,0.0,0.0,0.0,0.0,0.0,0.0,0,False,False,True
3,1968-01-08,16.0,NaN,NaN,NaN,NaN,<NA>,<NA>,1,64,...,0.0,0.0,0.0,0.0,0.0,0.0,0,False,False,True
4,1968-01-08,-16.0,NaN,NaN,NaN,NaN,<NA>,<NA>,0,64,...,0.0,0.0,0.0,0.0,0.0,0.0,1,False,False,True


In [19]:
df_train.describe()

,date,diff_elo,diff_rank,diff_points,diff_age,diff_height,p1_is_lefty,p2_is_lefty,same_country,draw_size,...,h2h_matches,p1_h2h_ace,p1_h2h_df,p1_h2h_1stWon,p1_h2h_bpSaved,p2_h2h_ace,p2_h2h_df,p2_h2h_1stWon,p2_h2h_bpSaved,target
count,188161,188161.000000,140921.000000,103631.000000,182601.000000,153371.000000,184156.0,161689.0,188161.000000,188161.000000,...,188161.000000,144571.000000,144571.000000,188161.000000,144571.000000,144571.000000,144571.000000,188161.000000,144571.000000,188161.000000
mean,1993-09-29 06:54:48.082014720,0.204262,-0.733375,-0.542907,-0.011975,-0.011951,0.142596,0.159516,0.130750,52.926292,...,1.238317,1.471359,0.794616,0.142787,1.094649,1.467327,0.792615,0.142759,1.091587,0.500805
min,1968-01-08 00:00:00,-971.289327,-2090.000000,-16304.000000,-41.700000,-41.000000,0.0,0.0,0.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1980-04-14 00:00:00,-130.119272,-45.000000,-479.500000,-3.600000,-5.000000,0.0,0.0,0.000000,32.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1993-03-01 00:00:00,0.000000,-1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,32.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,2006-07-21 00:00:00,131.004661,44.000000,482.000000,3.600000,5.000000,0.0,0.0,0.000000,64.000000,...,1.000000,0.500000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,1.000000
max,2022-11-27 00:00:00,991.269166,2125.000000,16641.000000,39.200000,43.000000,1.0,1.0,1.000000,128.000000,...,58.000000,75.000000,20.000000,1.000000,23.000000,61.000000,26.000000,1.000000,28.000000,1.000000
std,NaN,211.481505,154.714995,1810.805340,5.516120,9.076592,0.349662,0.366158,0.337127,36.446303,...,2.517880,3.383619,1.699356,0.287239,2.286921,3.382138,1.696068,0.287207,2.277119,0.500001


In [20]:
print(df_train.isna().sum())

date                   0
diff_elo               0
diff_rank          47240
diff_points        84530
diff_age            5560
diff_height        34790
p1_is_lefty         4005
p2_is_lefty        26472
same_country           0
draw_size              0
round                  0
best_of                0
tourney_level          0
p1_ace            119372
p1_df             119372
p1_1stWon              0
p1_bpSaved        119372
p2_ace            119420
p2_df             119420
p2_1stWon              0
p2_bpSaved        119420
h2h_matches            0
p1_h2h_ace         43590
p1_h2h_df          43590
p1_h2h_1stWon          0
p1_h2h_bpSaved     43590
p2_h2h_ace         43590
p2_h2h_df          43590
p2_h2h_1stWon          0
p2_h2h_bpSaved     43590
target                 0
surface_Clay           0
surface_Grass          0
surface_Hard           0
dtype: int64


## 3. Training and Validation

Since this is time-series data, DO NOT use random `train_test_split`. It must be splitted by time to simulate reality.

**Training**: Matches from 2000 to 2021.  
**Test**: Matches from 2022.

If it is mixed randomly, the model could learn from the "future" (e.g., use a match from 2022 to predict one from 2021), which will give a false accuracy.

In [21]:
cutting_date = pd.to_datetime('20211201', format='%Y%m%d')
X_train = df_train[df_train['date'] < cutting_date].drop(columns=['target', 'date'])
y_train = df_train[df_train['date'] < cutting_date]['target']
X_test = df_train[df_train['date'] >= cutting_date].drop(columns=['target', 'date'])
y_test = df_train[df_train['date'] >= cutting_date]['target']
print(f"Training matches (History): {len(X_train)}")
print(f"Testing matches (2022): {len(X_test)}")

Training matches (History): 185234
Testing matches (2022): 2927


In [22]:
model = RandomForestClassifier(
    n_estimators=100,   # Number of trees in the forest
    max_depth=5,        # Maximum depth of the tree to avoid overfitting
    random_state=42     # Seed for reproducibility
)

print("Training model...")
model.fit(X_train, y_train)
print("Model trained")

y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy (2022 matches): {acc:.2f}")

Training model...
Model trained
Test Accuracy (2022 matches): 0.66


In [ ]:
# Season starting in december
# Logic: Year + 1 if the month is >= 12 (December)
df_seasoned = df_train.copy()
df_seasoned['season_id'] = df_seasoned['date'].dt.year + (df_seasoned['date'].dt.month >= 12).astype(int)
print("Seasons identified:", df_seasoned['season_id'].unique())

def season_splitter(df, season_col):
    """
    Generator that returns indices (train, test) based on complete seasons.
    Training: Expansive (accumulates past seasons).
    Test: The immediately following season.
    """
    seasons = sorted(df[season_col].unique())
    if len(seasons) < 2:
        raise ValueError("Need at least 2 complete seasons in df_train to validate.")
    
    for i in range(1, len(seasons)):
        past_seasons = seasons[:i]
        current_season = seasons[i]
        
        train_idx = df[df[season_col].isin(past_seasons)].index.values
        test_idx = df[df[season_col] == current_season].index.values
        yield (train_idx, test_idx)

cols_exclude = ['target', 'date', 'season_id']
X = df_seasoned.copy().drop(columns=cols_exclude)
y = df_seasoned['target'].copy()

model = RandomForestClassifier()
cv_splits = season_splitter(df_seasoned, 'season_id')
scores = cross_val_score(model, X, y, cv=cv_splits, scoring='accuracy')

# Results
print("\n--- Results ---")
for i, score in enumerate(scores):
    print(f"Split {i+1} (Validate in season {sorted(df_seasoned['season_id'].unique())[i+1]}): {score:.2%}")
print(f"\nMean Accuracy: {scores.mean():.2%}")

Seasons identified: [1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981
 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995
 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009
 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022]

--- Results ---
Split 1 (Validate in season 1969): 69.01%
Split 2 (Validate in season 1970): 71.73%
Split 3 (Validate in season 1971): 70.46%
Split 4 (Validate in season 1972): 68.81%
Split 5 (Validate in season 1973): 66.91%
Split 6 (Validate in season 1974): 71.01%
Split 7 (Validate in season 1975): 71.14%
Split 8 (Validate in season 1976): 69.72%
Split 9 (Validate in season 1977): 69.09%
Split 10 (Validate in season 1978): 68.29%
Split 11 (Validate in season 1979): 69.74%
Split 12 (Validate in season 1980): 69.53%
Split 13 (Validate in season 1981): 67.65%
Split 14 (Validate in season 1982): 66.97%
Split 15 (Validate in season 1983): 62.88%
Split 16 (Validate in season 1984): 63.78%
Spli